<a href="https://colab.research.google.com/github/pokerface1o1/vae-gan/blob/main/vae_gan.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip3 install http://download.pytorch.org/whl/cu80/torch-0.3.1-cp36-cp36m-linux_x86_64.whl 
!pip3 install torchvision
!pip3 install visdom
!pip3 install pycrayon

    100% |████████████████████████████████| 496.4MB 49.8MB/s 
    100% |████████████████████████████████| 51kB 1.8MB/s 
    99% |████████████████████████████████| 5.8MB 40.4MB/s eta 0:00:01

    100% |████████████████████████████████| 5.9MB 192kB/s 
  Found existing installation: Pillow 4.0.0
    Uninstalling Pillow-4.0.0:
      Successfully uninstalled Pillow-4.0.0
    100% |████████████████████████████████| 51kB 1.9MB/s 
  Running setup.py bdist_wheel for visdom ... - \ done
  Stored in directory: /content/.cache/pip/wheels/d3/45/ad/a5b60260d8e6122de5cbafd192ba55aa61d9b23c3b43aff34c
  Running setup.py bdist_wheel for torchfile ... - done
  Stored in directory: /content/.cache/pip/wheels/27/0a/74/650124cbc320716ff2a6566e711a0c84f871647b33ffc868f6
Successfully built visdom torchfile


In [ ]:
from __future__ import print_function
import argparse
import os
import random
import math
import torch
import torch.nn as nn
import torch.legacy.nn as lnn
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.optim as optim
import torch.utils.data
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torchvision.utils as vutils
#import visdom
from torch.autograd import Variable
import torch.nn.functional as F
from google.colab import files

#vis = visdom.Visdom()
#vis.env = 'vae_dcgan'

batch_size = 100
nz = 100
niter = 25

transform = transforms.Compose([transforms.ToTensor(),
                                transforms.Normalize((0.5, 0.5, 0.5),
                                (0.5, 0.5, 0.5))])


trainset = dset.MNIST(root='../data', train=True,
                                     download=True,
                                     transform=transform)

testset = dset.MNIST(root='../data', train=False,
                                     download=True,
                                     transform=transform)

trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                         shuffle=True, num_workers=2)

testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                        shuffle=False, num_workers=2)



cudnn.benchmark = True

class Sampler(nn.Module):
    def __init__(self):
        super(Sampler, self).__init__()
        
    def forward(self,input):
        #mu = input[0]
        #logvar = input[1]
        mu, logvar = torch.chunk(input, 2, dim=1)
        
        std = logvar.mul(0.5).exp_() #calculate the STDEV
        #if opt.cuda:
        eps = torch.cuda.FloatTensor(std.size()).normal_() #random normalized noise
        #else:
        #eps = torch.FloatTensor(std.size()).normal_() #random normalized noise
        eps = Variable(eps)
        return eps.mul(std).add_(mu)



class Encoder(nn.Module):
    def __init__(self, image_size=784, h_dim=400, z_dim=nz):
        super(Encoder, self).__init__()
        self.encoder1 = nn.Sequential(
            nn.Conv2d(1, 16, kernel_size=5, padding=2),
            nn.BatchNorm2d(16),
            nn.LeakyReLU(0.2),
            nn.MaxPool2d(2),
            nn.Conv2d(16, 32, kernel_size=5, padding=2),
            nn.BatchNorm2d(32),
            nn.LeakyReLU(0.2),
            nn.MaxPool2d(2))
        self.encoder2 = nn.Sequential(
            nn.Linear(7*7*32, h_dim),
            nn.Linear(h_dim, z_dim*2))  # 2 for mean and variance.
    """    
    def reparametrize(self, mu, log_var):
        \"""z = mean + eps * sigma where eps is sampled from N(0, 1).\"""
        eps = Variable(torch.randn(mu.size(0), mu.size(1)))
        z = mu + eps * torch.exp(log_var/2)  # 2 for convert var to std
        return z
    """
    
    def forward(self, x):
        out = self.encoder1(x)
        out = out.view(out.size(0), -1)
        out = self.encoder2(out)
          # mean and log variance.
        #z = self.reparametrize(mu, log_var)    
        return out


class Decoder(nn.Module):
    def __init__(self, image_size=784, h_dim=400, z_dim=nz):
        super(Decoder, self).__init__()
        self.decoder1 = nn.Sequential(
            nn.Linear(z_dim, h_dim),
            nn.LeakyReLU(0.2),
            nn.Linear(h_dim, 7*7*32))
        self.decoder2 = nn.Sequential(
            nn.ConvTranspose2d(32, 16, kernel_size=4, padding=1, stride=2),
            nn.BatchNorm2d(16),
            nn.ConvTranspose2d(16, 1, kernel_size=4, padding=1, stride=2),
            nn.BatchNorm2d(1),
            nn.Tanh())
        
    def forward(self, x):
        out = self.decoder1(x)
        out = out.view(out.size(0), 32, 7, 7)
        out = self.decoder2(out)
        
        return out

class VAE(nn.Module):
    def __init__(self, image_size=784, h_dim=400, z_dim=nz):
        super(VAE, self).__init__()
        self.encoder = Encoder()
        self.sampler = Sampler()
        self.decoder = Decoder()
                     
    def forward(self,x):
        out = self.encoder(x)
        out = self.sampler(out)
        out = self.decoder(out)
        
        return out
    
    def make_cuda(self):
        self.encoder.cuda()
        self.sampler.cuda()
        self.decoder.cuda()

class ImprovedGAN_Discriminator(nn.Module):

    def __init__(self, featmap_dim=512, n_channel=1, use_gpu=True,
                 n_B=128, n_C=16):
        """
        Minibatch discrimination: learn a tensor to encode side information
        from other examples in the same minibatch.
        """
        super(ImprovedGAN_Discriminator, self).__init__()
        self.use_gpu = use_gpu
        self.n_B = n_B
        self.n_C = n_C
        self.featmap_dim = featmap_dim

        self.conv1 = nn.Conv2d(n_channel, int(featmap_dim / 4), 5,
                               stride=2, padding=2)

        self.conv2 = nn.Conv2d(int(featmap_dim / 4), int(featmap_dim / 2), 5,
                               stride=2, padding=2)
        self.BN2 = nn.BatchNorm2d(int(featmap_dim / 2))

        self.conv3 = nn.Conv2d(int(featmap_dim / 2), featmap_dim, 5,
                               stride=2, padding=2)
        self.BN3 = nn.BatchNorm2d(featmap_dim)

        T_ten_init = torch.randn(featmap_dim * 4 * 4, n_B * n_C) * 0.1
        self.T_tensor = nn.Parameter(T_ten_init, requires_grad=True)
        self.fc = nn.Linear(featmap_dim * 4 * 4 + n_B, 1)

    def forward(self, x):
        """
        Architecture is similar to DCGANs
        Add minibatch discrimination => Improved GAN.
        """
        x = F.leaky_relu(self.conv1(x), negative_slope=0.2)
        x = F.leaky_relu(self.BN2(self.conv2(x)), negative_slope=0.2)
        x = F.leaky_relu(self.BN3(self.conv3(x)), negative_slope=0.2)
        x = x.view(-1, self.featmap_dim * 4 * 4)

        # #### Minibatch Discrimination ###
        T_tensor = self.T_tensor
        if self.use_gpu:
            T_tensor = T_tensor.cuda()

        Ms = x.mm(T_tensor)
        Ms = Ms.view(-1, self.n_B, self.n_C)

        out_tensor = []
        for i in range(Ms.size()[0]):

            out_i = None
            for j in range(Ms.size()[0]):
                o_i = torch.sum(torch.abs(Ms[i, :, :] - Ms[j, :, :]), 1)
                o_i = torch.exp(-o_i)
                if out_i is None:
                    out_i = o_i
                else:
                    out_i = out_i + o_i

            out_tensor.append(out_i)

        out_T = torch.cat(tuple(out_tensor)).view(Ms.size()[0], self.n_B)
        x = torch.cat((x, out_T), 1)
        # #### Minibatch Discrimination ###

        x = F.sigmoid(self.fc(x))

        return x

    

netG = VAE()
netD = ImprovedGAN_Discriminator()
criterion = nn.BCELoss()
MSECriterion = nn.MSELoss()

input = torch.FloatTensor(batch_size, 1, 28, 28)
noise = torch.FloatTensor(2*((torch.randn(batch_size, nz).bernoulli_(0.5))-0.5))
fixed_noise = torch.FloatTensor(2*((torch.randn(batch_size, nz).bernoulli_(0.5))-0.5))
label = torch.FloatTensor(batch_size)
#real_label = 1
#fake_label = 0

print("CU:" + str(torch.cuda.is_available()))
netD.cuda()
netG.make_cuda()
criterion.cuda()
MSECriterion.cuda()
input, label = input.cuda(), label.cuda()
noise, fixed_noise = noise.cuda(), fixed_noise.cuda()

input = Variable(input)
label = Variable(label)
noise = Variable(noise)
fixed_noise = Variable(fixed_noise)

optimizerD = optim.Adam(netD.parameters(), lr=0.0002, betas=(0.5, 0.999))
optimizerG = optim.Adam(netG.parameters(), lr=0.0002, betas=(0.5, 0.999))

gen_win = None
rec_win = None

for epoch in range(niter):
    for i, data in enumerate(trainloader, 0):
        netD.zero_grad()
        real_cpu, _ = data
        batch_size = real_cpu.size(0)
        input.data.resize_(real_cpu.size()).copy_(real_cpu)
        real_label = random.uniform(0.6,1.2)
        label.data.resize_(real_cpu.size(0)).fill_(real_label)
        
        output = netD(input)
        errD_real = criterion(output, label)
        errD_real.backward()
        D_x = output.data.mean()
        noise.data.resize_(batch_size, nz)
        noise.data.normal_(0, 1)
        gen = netG.decoder(noise)
        #print(gen.size())
        #gen_win = gen.data.numpy()
        if i%50==0:
            vutils.save_image(gen.data*0.5+0.5, './noise_'+str(epoch)+'_'+str(i)+'.png')
            #files.download('./noise_'+str(epoch)+'_'+str(i)+'.png')
        #gen_win = vis.image(gen.data[0].cpu()*0.5+0.5,win = gen_win)
        #print(gen.size())
        fake_label = random.uniform(0.0,0.3)
        label.data.fill_(fake_label)
        output = netD(gen.detach())
        errD_fake = criterion(output, label)
        errD_fake.backward()
        D_G_z1 = output.data.mean()
        errD = errD_real + errD_fake
        optimizerD.step()
        
        #print(input.size())
        encoded = netG.encoder(input)
        mu, logvar = torch.chunk(encoded, 2, dim=1)
        
        KLD_element = mu.pow(2).add_(logvar.exp()).mul_(-1).add_(1).add_(logvar)
        KLD = torch.sum(KLD_element).mul_(-0.5)
        
        sampled = netG.sampler(encoded)
        rec = netG.decoder(sampled)
        #rec_win = rec.data.numpy()
        if i%50==0:
            vutils.save_image(rec.data*0.5+0.5, './sample_'+str(epoch)+'_'+str(i)+'.png')
            #files.download('./sample_'+str(epoch)+'_'+str(i)+'.png')
        #rec_win = vis.image(rec.data[0].cpu()*0.5+0.5,win = rec_win)
        
        MSEerr = MSECriterion(rec,input)
        
        VAEerr = KLD + MSEerr
        VAEerr.backward()
        optimizerG.step()
        
        netG.zero_grad()
        real_label = random.uniform(0.6,1.2)
        label.data.fill_(real_label)  # fake labels are real for generator cost

        rec = netG(input) # this tensor is freed from mem at this point
        output = netD(rec)
        errG = criterion(output, label)
        errG.backward()
        D_G_z2 = output.data.mean()
        optimizerG.step()

        print('[%d/%d][%d/%d] Loss_VAE: %.4f Loss_D: %.4f Loss_G: %.4f D(x): %.4f D(G(z)): %.4f / %.4f'
              % (epoch, niter, i, len(trainloader),
                 VAEerr.data[0], errD.data[0], errG.data[0], D_x, D_G_z1, D_G_z2))
        
        if(epoch!=0):
          torch.save(netG, './g' )
          torch.save(netD, './d')


CU:True


/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:1189: UserWarning: Using a target size (torch.Size([100])) that is different to the input size (torch.Size([100, 1])) is deprecated. Please ensure they have the same size.
  "Please ensure they have the same size.".format(target.size(), input.size()))


[0/25][0/600] Loss_VAE: 387.4487 Loss_D: 1.4017 Loss_G: 1.4593 D(x): 0.4855 D(G(z)): 0.4878 / 0.2614
[0/25][1/600] Loss_VAE: 166.8496 Loss_D: 0.1268 Loss_G: 0.9465 D(x): 0.9713 D(G(z)): 0.3710 / 0.3987
[0/25][2/600] Loss_VAE: 117.1510 Loss_D: 3.1381 Loss_G: 1.8336 D(x): 0.9959 D(G(z)): 0.6110 / 0.1289
[0/25][3/600] Loss_VAE: 97.5845 Loss_D: 1.1811 Loss_G: 2.7654 D(x): 0.9723 D(G(z)): 0.2891 / 0.0746
[0/25][4/600] Loss_VAE: 97.7590 Loss_D: 0.9923 Loss_G: 1.1530 D(x): 0.7546 D(G(z)): 0.2022 / 0.3350
[0/25][5/600] Loss_VAE: 98.0717 Loss_D: 1.2568 Loss_G: 1.6834 D(x): 0.7348 D(G(z)): 0.5652 / 0.1185
[0/25][6/600] Loss_VAE: 87.1173 Loss_D: 0.6899 Loss_G: 1.8499 D(x): 0.7065 D(G(z)): 0.2019 / 0.1478
[0/25][7/600] Loss_VAE: 79.0588 Loss_D: -0.1132 Loss_G: 2.1478 D(x): 0.9586 D(G(z)): 0.2651 / 0.0488
[0/25][8/600] Loss_VAE: 75.6929 Loss_D: 1.2243 Loss_G: 1.1602 D(x): 0.9888 D(G(z)): 0.1166 / 0.2772
[0/25][9/600] Loss_VAE: 69.6176 Loss_D: 0.6941 Loss_G: 3.7507 D(x): 0.9907 D(G(z)): 0.5148 / 0.0

[0/25][44/600] Loss_VAE: 33.4761 Loss_D: 1.0257 Loss_G: 1.7302 D(x): 0.5047 D(G(z)): 0.0614 / 0.0968
[0/25][45/600] Loss_VAE: 34.2090 Loss_D: 0.9235 Loss_G: 2.0127 D(x): 0.8025 D(G(z)): 0.0998 / 0.1115
[0/25][46/600] Loss_VAE: 31.7513 Loss_D: 0.8875 Loss_G: 1.7046 D(x): 0.8627 D(G(z)): 0.1144 / 0.0778
[0/25][47/600] Loss_VAE: 26.6526 Loss_D: 0.8464 Loss_G: 2.2434 D(x): 0.8218 D(G(z)): 0.0802 / 0.0565
[0/25][48/600] Loss_VAE: 23.0641 Loss_D: 1.4014 Loss_G: 1.8391 D(x): 0.7460 D(G(z)): 0.0576 / 0.1071
[0/25][49/600] Loss_VAE: 20.0070 Loss_D: 0.9478 Loss_G: 1.7828 D(x): 0.7585 D(G(z)): 0.1093 / 0.1685
[0/25][50/600] Loss_VAE: 19.8107 Loss_D: 0.6274 Loss_G: 0.9711 D(x): 0.7795 D(G(z)): 0.1730 / 0.3763
[0/25][51/600] Loss_VAE: 18.5235 Loss_D: 1.4133 Loss_G: 2.0229 D(x): 0.9033 D(G(z)): 0.3836 / 0.1368
[0/25][52/600] Loss_VAE: 20.0389 Loss_D: 1.2321 Loss_G: 2.2211 D(x): 0.8337 D(G(z)): 0.1391 / 0.0929
[0/25][53/600] Loss_VAE: 21.7364 Loss_D: 0.9136 Loss_G: 2.5734 D(x): 0.6978 D(G(z)): 0.0943

[0/25][87/600] Loss_VAE: 25.1502 Loss_D: 0.5214 Loss_G: 1.6501 D(x): 0.9179 D(G(z)): 0.2024 / 0.1852
[0/25][88/600] Loss_VAE: 34.3492 Loss_D: -0.1244 Loss_G: 1.9373 D(x): 0.9297 D(G(z)): 0.1906 / 0.1592
[0/25][89/600] Loss_VAE: 41.3276 Loss_D: 0.7357 Loss_G: 2.5868 D(x): 0.9548 D(G(z)): 0.1646 / 0.0953
[0/25][90/600] Loss_VAE: 34.5333 Loss_D: 0.8559 Loss_G: 2.0359 D(x): 0.9364 D(G(z)): 0.0957 / 0.1410
[0/25][91/600] Loss_VAE: 20.2819 Loss_D: -0.2370 Loss_G: 1.0702 D(x): 0.9623 D(G(z)): 0.1438 / 0.2013
[0/25][92/600] Loss_VAE: 12.3178 Loss_D: 1.3144 Loss_G: 1.2612 D(x): 0.9795 D(G(z)): 0.2091 / 0.1605
[0/25][93/600] Loss_VAE: 10.8377 Loss_D: 0.2484 Loss_G: 1.8546 D(x): 0.9653 D(G(z)): 0.1650 / 0.1500
[0/25][94/600] Loss_VAE: 11.6959 Loss_D: 0.6052 Loss_G: 2.5675 D(x): 0.9771 D(G(z)): 0.1514 / 0.0883
[0/25][95/600] Loss_VAE: 11.6096 Loss_D: 0.8693 Loss_G: 3.2656 D(x): 0.9576 D(G(z)): 0.0879 / 0.0557
[0/25][96/600] Loss_VAE: 11.7349 Loss_D: 0.8228 Loss_G: 2.1891 D(x): 0.9379 D(G(z)): 0.05

[0/25][130/600] Loss_VAE: 11.8196 Loss_D: 0.8225 Loss_G: 2.1035 D(x): 0.7993 D(G(z)): 0.0905 / 0.1193
[0/25][131/600] Loss_VAE: 14.4376 Loss_D: 0.8935 Loss_G: 2.7731 D(x): 0.8285 D(G(z)): 0.1159 / 0.0893
[0/25][132/600] Loss_VAE: 15.9844 Loss_D: 0.2972 Loss_G: 2.4584 D(x): 0.7809 D(G(z)): 0.0895 / 0.1027
[0/25][133/600] Loss_VAE: 16.1199 Loss_D: 0.5836 Loss_G: 1.4292 D(x): 0.8598 D(G(z)): 0.1017 / 0.2146
[0/25][134/600] Loss_VAE: 13.8094 Loss_D: 0.8250 Loss_G: 1.3990 D(x): 0.9248 D(G(z)): 0.2225 / 0.2401
[0/25][135/600] Loss_VAE: 10.9623 Loss_D: 0.1885 Loss_G: 0.9480 D(x): 0.9516 D(G(z)): 0.2463 / 0.2648
[0/25][136/600] Loss_VAE: 10.0425 Loss_D: 0.3689 Loss_G: 0.9067 D(x): 0.9676 D(G(z)): 0.2642 / 0.3478
[0/25][137/600] Loss_VAE: 9.0631 Loss_D: 0.8753 Loss_G: 1.5323 D(x): 0.9822 D(G(z)): 0.3569 / 0.2160
[0/25][138/600] Loss_VAE: 9.0357 Loss_D: 0.2564 Loss_G: 2.3168 D(x): 0.9816 D(G(z)): 0.2206 / 0.0828
[0/25][139/600] Loss_VAE: 8.9833 Loss_D: -0.1856 Loss_G: 2.9721 D(x): 0.9728 D(G(z))

[0/25][173/600] Loss_VAE: 5.7290 Loss_D: 1.0707 Loss_G: 2.0270 D(x): 0.8875 D(G(z)): 0.1991 / 0.1288
[0/25][174/600] Loss_VAE: 6.2417 Loss_D: 0.5054 Loss_G: 1.7249 D(x): 0.8151 D(G(z)): 0.1404 / 0.1282
[0/25][175/600] Loss_VAE: 6.3024 Loss_D: 0.8946 Loss_G: 2.4024 D(x): 0.8416 D(G(z)): 0.1387 / 0.1319
[0/25][176/600] Loss_VAE: 6.5499 Loss_D: 1.5000 Loss_G: 1.9963 D(x): 0.8656 D(G(z)): 0.1380 / 0.1424
[0/25][177/600] Loss_VAE: 7.0755 Loss_D: 0.3415 Loss_G: 1.8854 D(x): 0.8078 D(G(z)): 0.1529 / 0.1526
[0/25][178/600] Loss_VAE: 7.3964 Loss_D: 0.7413 Loss_G: 2.0748 D(x): 0.8584 D(G(z)): 0.1594 / 0.1104
[0/25][179/600] Loss_VAE: 8.0782 Loss_D: 1.4793 Loss_G: 2.4599 D(x): 0.8092 D(G(z)): 0.1071 / 0.1152
[0/25][180/600] Loss_VAE: 8.8513 Loss_D: 0.3844 Loss_G: 1.2551 D(x): 0.7717 D(G(z)): 0.1238 / 0.1489
[0/25][181/600] Loss_VAE: 9.6621 Loss_D: 1.1522 Loss_G: 2.1953 D(x): 0.8165 D(G(z)): 0.1550 / 0.1110
[0/25][182/600] Loss_VAE: 10.1755 Loss_D: 1.3864 Loss_G: 2.0664 D(x): 0.7786 D(G(z)): 0.127

[0/25][216/600] Loss_VAE: 7.9726 Loss_D: 0.4023 Loss_G: 2.0434 D(x): 0.8470 D(G(z)): 0.1522 / 0.1765
[0/25][217/600] Loss_VAE: 8.2506 Loss_D: 0.2242 Loss_G: 1.3093 D(x): 0.8842 D(G(z)): 0.1840 / 0.2219
[0/25][218/600] Loss_VAE: 8.3333 Loss_D: 0.2160 Loss_G: 0.8283 D(x): 0.9344 D(G(z)): 0.2196 / 0.3345
[0/25][219/600] Loss_VAE: 8.0495 Loss_D: 0.4526 Loss_G: 1.4252 D(x): 0.9662 D(G(z)): 0.3534 / 0.2728
[0/25][220/600] Loss_VAE: 7.5179 Loss_D: -0.3246 Loss_G: 2.0845 D(x): 0.9737 D(G(z)): 0.2744 / 0.1412
[0/25][221/600] Loss_VAE: 6.9261 Loss_D: 1.3421 Loss_G: 2.7383 D(x): 0.9780 D(G(z)): 0.1476 / 0.0761
[0/25][222/600] Loss_VAE: 6.8564 Loss_D: 1.1367 Loss_G: 2.6137 D(x): 0.9594 D(G(z)): 0.0747 / 0.0831
[0/25][223/600] Loss_VAE: 6.7043 Loss_D: 1.1798 Loss_G: 2.2397 D(x): 0.9547 D(G(z)): 0.0881 / 0.1045
[0/25][224/600] Loss_VAE: 7.2035 Loss_D: 0.7451 Loss_G: 1.6327 D(x): 0.9002 D(G(z)): 0.1064 / 0.1463
[0/25][225/600] Loss_VAE: 8.1163 Loss_D: 1.2548 Loss_G: 2.1088 D(x): 0.9216 D(G(z)): 0.143

[0/25][259/600] Loss_VAE: 7.0030 Loss_D: 0.7216 Loss_G: 1.6195 D(x): 0.8584 D(G(z)): 0.1603 / 0.2365
[0/25][260/600] Loss_VAE: 7.1947 Loss_D: 0.9935 Loss_G: 1.3457 D(x): 0.9055 D(G(z)): 0.2400 / 0.2116
[0/25][261/600] Loss_VAE: 6.6331 Loss_D: 0.5481 Loss_G: 1.2517 D(x): 0.9015 D(G(z)): 0.2163 / 0.1449
[0/25][262/600] Loss_VAE: 7.1066 Loss_D: 0.2217 Loss_G: 1.5580 D(x): 0.8788 D(G(z)): 0.1412 / 0.1101
[0/25][263/600] Loss_VAE: 7.9543 Loss_D: 0.7852 Loss_G: 2.1499 D(x): 0.8635 D(G(z)): 0.1150 / 0.1322
[0/25][264/600] Loss_VAE: 8.4122 Loss_D: 0.4653 Loss_G: 1.3932 D(x): 0.9058 D(G(z)): 0.1450 / 0.2141
[0/25][265/600] Loss_VAE: 7.5363 Loss_D: 1.3769 Loss_G: 1.4514 D(x): 0.9503 D(G(z)): 0.2156 / 0.2722
[0/25][266/600] Loss_VAE: 7.5175 Loss_D: 1.0544 Loss_G: 2.2168 D(x): 0.9558 D(G(z)): 0.2859 / 0.1504
[0/25][267/600] Loss_VAE: 7.6800 Loss_D: 0.4467 Loss_G: 1.5727 D(x): 0.9197 D(G(z)): 0.1510 / 0.0847
[0/25][268/600] Loss_VAE: 7.6741 Loss_D: 1.0712 Loss_G: 2.7978 D(x): 0.8948 D(G(z)): 0.0861

[0/25][302/600] Loss_VAE: 6.0620 Loss_D: 1.3728 Loss_G: 2.4305 D(x): 0.9461 D(G(z)): 0.2095 / 0.1294
[0/25][303/600] Loss_VAE: 5.5230 Loss_D: 0.3828 Loss_G: 2.2792 D(x): 0.8975 D(G(z)): 0.1282 / 0.0908
[0/25][304/600] Loss_VAE: 5.0549 Loss_D: 0.5773 Loss_G: 2.1980 D(x): 0.8453 D(G(z)): 0.0913 / 0.0777
[0/25][305/600] Loss_VAE: 5.4772 Loss_D: 0.8470 Loss_G: 1.7428 D(x): 0.8515 D(G(z)): 0.0759 / 0.0909
[0/25][306/600] Loss_VAE: 5.5802 Loss_D: 0.5675 Loss_G: 1.5202 D(x): 0.8488 D(G(z)): 0.0861 / 0.1436
[0/25][307/600] Loss_VAE: 5.7244 Loss_D: 0.1512 Loss_G: 1.8024 D(x): 0.9260 D(G(z)): 0.1400 / 0.1549
[0/25][308/600] Loss_VAE: 5.9396 Loss_D: 1.0377 Loss_G: 1.8168 D(x): 0.9322 D(G(z)): 0.1567 / 0.1260
[0/25][309/600] Loss_VAE: 5.6828 Loss_D: 0.3620 Loss_G: 2.0349 D(x): 0.9258 D(G(z)): 0.1295 / 0.1014
[0/25][310/600] Loss_VAE: 5.2701 Loss_D: -0.0604 Loss_G: 2.4128 D(x): 0.9363 D(G(z)): 0.1014 / 0.0967


In [ ]:
!pwd

/content


In [ ]:
!ls datalab

In [ ]:
!ls

datalab


In [ ]:
import os 
os.mkdir('/content/output')

In [ ]:
!pwd

/content


In [ ]:
!cd /content/output

In [ ]:
!pwd

/content


In [ ]:
!ls

datalab  output


In [ ]:
!cd output

In [ ]:
!pwd

/content
